# EDINETに存在するドキュメント一覧を取得する
過去10年分のデータ一覧を取得する

参考サイト:https://www.quwechan.com/entry/2023/01/25/180401

In [2]:
import requests
import time

## sample
指定した日付に提出された書類の一覧を取得する

In [8]:
#requestsの設定
#typeは1,2があり，1はメタデータのみ，2は提出証類一覧及び，メタデータを取得する
#dateはYYYY-MM-DD形式で指定する
params = {
	'type' : 2,
	'date' : '2023-01-25'
}

#requestsの実行
res = requests.get('https://disclosure.edinet-fsa.go.jp/api/v1/documents.json', \
			params=params, \
			verify=False, \
			timeout = 120.0)

/Users/momoidaisuke/practice/env/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
print(res.text)

{
  "metadata": {
    "title": "提出された書類を把握するためのAPI",
    "parameter": {
      "date": "2023-01-25",
      "type": "2"
    },
    "resultset": {
      "count": 188
    },
    "processDateTime": "2023-03-28 00:04",
    "status": "200",
    "message": "OK"
  },
  "results": [
    {
      "seqNumber": 1,
      "docID": "S100PZZF",
      "edinetCode": "E10677",
      "secCode": null,
      "JCN": "9010001021473",
      "filerName": "アセットマネジメントＯｎｅ株式会社",
      "fundCode": "G14746",
      "ordinanceCode": "030",
      "formCode": "04A000",
      "docTypeCode": "030",
      "periodStart": null,
      "periodEnd": null,
      "submitDateTime": "2023-01-25 09:00",
      "docDescription": "有価証券届出書（内国投資信託受益証券）",
      "issuerEdinetCode": null,
      "subjectEdinetCode": null,
      "subsidiaryEdinetCode": null,
      "currentReportReason": null,
      "parentDocID": null,
      "opeDateTime": null,
      "withdrawalStatus": "0",
      "docInfoEditStatus": "0",
      "disclosureStatus": "0",
      "

## 関数化

In [10]:
import requests
import time
import json
import os
import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path
import sys

In [7]:
def get_edinet_json_text(date_str) :
	"""
	EDINETから指定日のデータを取得する
	Args:
		date_str (str): 取得する日付
	Returns:
		json_text (str): 取得したデータ
	"""
	json_text = None

	params = {
		'type' : 2,
		'date' : date_str
	}


	url = 'https://disclosure.edinet-fsa.go.jp/api/v1/documents.json'
	retry_count = -1
	
	# リクエストを10回まで送信する
	while retry_count < 10 :

		retry_count = retry_count + 1

		print(url + '?' + t_date + 'へリクエスト送信')

		time.sleep(2.0)
		res = requests.get(url, \
				params=params, \
				verify=False, \
				timeout = 120.0)

		# リクエストが失敗した場合は、60秒待って再度リクエストを送信する
		if res.status_code != 200 :

			print('status_code:' + res.status_code)
			res.close()
			time.sleep(60.0)
			continue
		
		# レスポンスのcontent-typeがjsonでない場合は、60秒待って再度リクエストを送信する	
		if res.headers['content-type'] != 'application/json; charset=utf-8' :

			print('content-type:' + res.headers['content-type'])
			res.close()
			time.sleep(60.0)
			continue

		
		json_text = res.text
		res.close()
		break

	return json_text

In [31]:
def get_edinet_document_list_json_dir_path() :
	"""
	EDINETのデータを保存するディレクトリのパスを取得する
	
	Returns:
		dir_path (Path): ディレクトリのパス
	"""

	return Path('../01_data/edinet_document_list_json')

In [32]:
def get_edinet_document_list_json_file_path(t_date) :
	"""
	EDINETのデータを保存するファイルのパスを取得する
	Args:
		t_date (_type_): _description_
	"""

	save_path = get_edinet_document_list_json_dir_path().joinpath(t_date + '.json' )
    
	return save_path

In [33]:
def save_edinet_document_list_json(t_date, json_text) :
	"""
	EDINETのデータを保存する
	Args:
		t_date (_type_): 日付
		json_text (_type_): 保存するJSONデータ
	Returns:
		_type_: _description_
	"""

	get_edinet_document_list_json_dir_path().mkdir(exist_ok = True, parents = True)

	save_path = get_edinet_document_list_json_file_path(t_date)

	value = json.loads(json_text)


	if value['metadata']['status'] != '200' :

		print(t_date + '.json status:' + value['metadata']['status'])
		sys.exit()

	file = open(save_path, "w")
	json.dump(value, file, indent = 4)
	file.close()


In [34]:
t_date = '2023-01-25'


print('edinetからドキュメントリストを取得')
json_text = get_edinet_json_text(t_date)


if not get_edinet_document_list_json_file_path(t_date).exists() :
	print(str(get_edinet_document_list_json_file_path(t_date)) +'を保存')
	save_edinet_document_list_json(t_date, json_text)

edinetからドキュメントリストを取得
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json?2023-01-25へリクエスト送信


/Users/momoidaisuke/practice/env/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


../01_data/edinet_document_list_json/2023-01-25.jsonを保存


## データの取得
10年分のデータを取得する

In [37]:
#過去10年分の日付を生成し、各日付について処理

d_now  = datetime.datetime.now()
ten_years_ago_d_date = d_now - relativedelta(years=10)
ten_years_ago_t_date = ten_years_ago_d_date.strftime('%Y-%m-%d')


for trace_back_days in range(3700, 2, -1) :

	# 日付を生成
	d_date = d_now - datetime.timedelta(days=trace_back_days)
	# 日付を文字列に変換
	t_date = d_date.strftime('%Y-%m-%d')

	# 10年前の日付より前の日付は処理しない
	if t_date < ten_years_ago_t_date :

		continue

	# すでにファイルが存在する場合は処理しない
	if get_edinet_document_list_json_file_path(t_date).exists() :

		print(str(get_edinet_document_list_json_file_path(t_date)) +' is already exists. skip')
		continue

	# ファイルが存在しない場合は、EDINETからデータを取得し、ファイルに保存する
	print(str(get_edinet_document_list_json_file_path(t_date))+' processing')
	# EDINETからデータを取得する
	json_text = get_edinet_json_text(t_date)

	# データが取得できなかった場合は、処理を中断する
	if json_text == None :

		print('json download fail. 処理中断')
		break

	# データをファイルに保存する
	save_edinet_document_list_json(t_date, json_text)

../01_data/edinet_document_list_json/2013-03-06.json processing
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json?2013-03-06へリクエスト送信


/Users/momoidaisuke/practice/env/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


../01_data/edinet_document_list_json/2013-03-07.json processing
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json?2013-03-07へリクエスト送信


/Users/momoidaisuke/practice/env/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


../01_data/edinet_document_list_json/2013-03-08.json processing
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json?2013-03-08へリクエスト送信


...

/Users/momoidaisuke/practice/env/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


../01_data/edinet_document_list_json/2023-03-02.json processing
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json?2023-03-02へリクエスト送信


/Users/momoidaisuke/practice/env/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


../01_data/edinet_document_list_json/2023-03-03.json processing
https://disclosure.edinet-fsa.go.jp/api/v1/documents.json?2023-03-03へリクエスト送信


/Users/momoidaisuke/practice/env/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
